---
# Archivo Properati.  
## Primera exploración sobre los datos

In [1]:
import pandas as pd
import numpy as np
import re

# importamos el archivo
data_location = "C:/Users/Fernando/Documents/DIGITAL_HOUSE/CONTENIDO/DESAFIO_I/properati.csv"
data_original = pd.read_csv(data_location, sep=",", encoding="utf-8")

# renombro la columna que no tiene nombre
data_original = data_original.rename({           
    'Unnamed: 0': 'Id_caso'
}, axis=1)

print('Variables y tipos de datos')
print('--------------------------')
# tipo de datos de cada columna
print (data_original.dtypes)

print('Abrimos y vemos los datos')
print('-------------------------')
# abrimos el archivo
print(data_original.head(1))

print ('Porcentaje de nulos por variable')
print ('--------------------------------')
# % de nulos por columna
print (round(data_original.isnull().sum() / data_original.shape[0]*100,1))

Variables y tipos de datos
--------------------------
Id_caso                         int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          obj

---
# Trabajo sobre variables

Las tareas sobre las variables las organizamos de la siguiente manera:
* Agrupamos cada una de acuerdo a criterios comunes

In [2]:
var = pd.DataFrame({'Grupo': [\
                              'Característica', 'Característica', 'Característica','Característica', 'Característica', 'Característica',\
                              'Localización','Localización','Localización','Localización','Localización','Localización','Localización','Localización',\
                              'Dimensión','Dimensión',\
                              'Valor','Valor','Valor','Valor','Valor','Valor',\
                              'Otras','Otras','Otras','Otras'\
                             ],
                    'Variable': ['operation', 'Tipo', 'floor','rooms', 'expenses', 'amenities',\
                                 'place_name','place_with_parent_names','country_name','state_name','geonames_id','lat-lon','lat','lon',\
                                 'surface_total_in_m2','surface_covered_in_m2',\
                                 'price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2',\
                                 'description','title','properati_url','image_thumbnail'\
                                ]})

print(var.groupby(["Grupo","Variable"])["Grupo"].count())

Grupo           Variable                  
Característica  Tipo                          1
                amenities                     1
                expenses                      1
                floor                         1
                operation                     1
                rooms                         1
Dimensión       surface_covered_in_m2         1
                surface_total_in_m2           1
Localización    country_name                  1
                geonames_id                   1
                lat                           1
                lat-lon                       1
                lon                           1
                place_name                    1
                place_with_parent_names       1
                state_name                    1
Otras           description                   1
                image_thumbnail               1
                properati_url                 1
                title                        

* De acuerdo a cada grupo, nos separamos su análisis pero definiendo objetivos generales:
    * Analisis de distribución.
    * Definición si la variable era necesaria o no.
    * Si alguna dependía o tenía su origen en otra.
    * Búsqueda de errores y tareas de corrección.
    * Política de llenado de valores nulos, si era posible.
    * Identificación y política de valores outliers.
* Trabajo de expresiones regulares para:
    * Identificar valores faltantes sobre variables existentes
    * Identificación de variables que sumen al objetivo.  Amenities.
* Búsqueda y eliminación de registros duplicados.
* Búsqueda y eliminación de registros únicos que finalmente no puedan ser utilizados para el cumplimiento del objetivo.
* Finalmente genearación de un nuevo DF.

---
# 1.  Variables del grupo de localización de propiedades


Validación, limpieza y asignación de nulos; sobre las variables:

- country_name 
- state_name
- place_name
- geonames_id
- place_with_parent_names
- lat-lon
- lat
- lon

  **country_name**

In [3]:
# Confirmamos que la variable tiene un solo valor, por lo que es posible descartarla
data_original.country_name.value_counts()

Argentina    121220
Name: country_name, dtype: int64

  **state_name**

In [4]:
# Distribución de la variable
print (data_original.state_name.value_counts())
print ("--------------------------------------------------")
print ("Vemos que no existen valores por fuera del listado")

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

  **place_name**
  
  Validamos si los datos de la variable están completos y es un dato confiable, o si debemos realizar alguna terea sobre ellos.
  * Luego del trabajo exploratorio, vamos a confirmar que place_name correponde a la última parte de la variable place_with_parent_names:
      * Separamos place_with_parent_names en 5 variables nuevas:
          * Pais
          * Zona
          * Partido_barrio
          * Localidad
          * Obs_localidad

In [5]:
# vamos a determinadar si la variable 'place_name' tiene un dato 'confiable' o 'completo', 
# para eso separamos la variable 'place_with_parent_names' en partes, por el simbolo | (pipe) 
# separo la variable place_with_parent_names, sumando campos nuevos y ya trabajando sobre le nuevo dataframe
data = data_original.join(data_original["place_with_parent_names"].str.split('|', expand=True).rename(columns={
                                                                                                1:'Pais', 
                                                                                                2:'Zona', 
                                                                                                3:'Partido_barrio', 
                                                                                                4:'Localidad', 
                                                                                                5:'Obs_localidad', 
                                                                                                6:'Descarte'}))
data.drop(['Descarte'], axis='columns', inplace=True)

* Se recorren las variables creadas desde la última hasta encontrar un NO nulo se graba el valor encontrado en place_name_new
* Se compara la variable creada vs place_name original

In [6]:
# validamos que la variable place_name sea igual a el ultimo valor informado sobre place_with_parent_names
# ya separado por el proceso anterior
# lo recorro desde la ultima variable hasta encontrar un NO nulo y grabo el valor
# 1 - Obs_localidad
# 2 - Localidad
# 3 - Partido_barrio
# 4 - Zona
# creo una nueva variable que voy a ir completando a medida que valido que cada 
# variable tenga valor empezando desde la última
data["place_name_new"] = 'validar'

#para cada variable nuevo valido null o ''
#---------------------------------------------------------------------------
# creo mascara de los casos donde la varible 'x' tenga null o ''
mask_data_Obs_localidad_null =  (data.Obs_localidad.isnull()) | (data.Obs_localidad == '')
# creo otra mascara con origen en la mascara enterior para quedarme con los 
# que tienen valor en la variable 'x'
mask_data_Obs_localidad_notnull = mask_data_Obs_localidad_null == False
print ('Actualización de place_name_new')
print ('--------------------------------')

print ('Obs_localidad')
print ('Cantidad de casos con Obs_localidad informada: ' + str(mask_data_Obs_localidad_notnull.sum()))
# creo una mascara sobre la variable que voy a ir completando con el valor obtenido
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original :' + str(mask_data_place_name_new_validar.sum()))
# actualizo los valores sobre mi nueva variable con los que encontré en la variable 'x'
data.loc[mask_data_Obs_localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data.Obs_localidad
# actualizo la mascara sobre mi nueva variable para contar y ver si todo resultó ok!
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
# la cantidad de casos sobre la variable nueva con valor 'validar' era de 121197
# La cantidad de casos sobre la misma variable con valor 'validar' despues de actualizar fue de 120649
# El delta corresponde a los valores con origen en el primer campo Obs_localidad ... 548

#---------------------------------------
print ('--------------------------------')
# 2 - Localidad
mask_data_Localidad_null =  (data.Localidad.isnull()) | (data.Localidad == '')
mask_data_Localidad_notnull = mask_data_Localidad_null == False
print ('Localidad')
print ('Cantidad de casos con Localidad informada: ' + str(mask_data_Localidad_notnull.sum()))
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' +  str(mask_data_place_name_new_validar.sum()))
data.loc[mask_data_Localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data.Localidad
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#---------------------------------------
print ('--------------------------------')
# 3 - Partido_barrio
mask_data_Partido_barrio_null =  (data.Partido_barrio.isnull()) | (data.Partido_barrio == '')
mask_data_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
print ('Partido_barrio')
print ('Cantidad de casos con Partido_barrio informada: ' + str(mask_data_Partido_barrio_notnull.sum()))
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))
data.loc[mask_data_Partido_barrio_notnull&mask_data_place_name_new_validar, "place_name_new"] = data.Partido_barrio
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#---------------------------------------
print ('--------------------------------')
# 4 - Zona
mask_data_Zona_null =  (data.Zona.isnull()) | (data.Zona == '')
mask_data_Zona_notnull = mask_data_Zona_null == False
print ('Zona')
print ('Cantidad de casos con Zona informada: ' + str(mask_data_Zona_notnull.sum()))
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))
data.loc[mask_data_Zona_notnull&mask_data_place_name_new_validar, "place_name_new"] = data.Zona
mask_data_place_name_new_validar = data.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))
#----------------------------------------------------
print ('--------------------------------')
print ('Cantidad casos que coinciden entre place_name_new y place_name')
# valido si place_name_new da igual que place_name, si contamos lo mismo es que da igual
mask_place_name_validacion =  data.place_name_new == data.place_name
print(mask_place_name_validacion.value_counts())  
mask_place_name_validacion_falso = mask_place_name_validacion == False
print ('Vemos que casuiticas encontramos sobre los casos que no encontramos:')
print(data.loc[mask_place_name_validacion_falso, ["Id_caso","place_with_parent_names","place_name", "place_name_new"]].\
     groupby(["place_with_parent_names"])["Id_caso"].count())
print('Vemos que son todos del mismo place_with_parent_names de Tigre')
# lo normalizamos
print ('Los normalizamos sobre la variable place_name_new')
data.loc[mask_place_name_validacion_falso, "place_name_new"] = 'Tigre'
data.loc[mask_place_name_validacion_falso, "Localidad"] = 'Tigre'
print ('Validamos la actualización:')
print(data.loc[mask_place_name_validacion_falso, ["Id_caso", "place_name_new"]]\
     .groupby(["place_name_new"])["Id_caso"].count())
print('----------------------------------------------------------------------------')
print('Confirmamos que la variable creada - place_name_new - es igual a  place_name')

Actualización de place_name_new
--------------------------------
Obs_localidad
Cantidad de casos con Obs_localidad informada: 548
Cantidad de casos con place_name_new original :121220
Cantidad de casos con place_name_new posterior para analizar: 120672
--------------------------------
Localidad
Cantidad de casos con Localidad informada: 40417
Cantidad de casos con place_name_new original: 120672
Cantidad de casos con place_name_new posterior para analizar: 80803
--------------------------------
Partido_barrio
Cantidad de casos con Partido_barrio informada: 116440
Cantidad de casos con place_name_new original: 80803
Cantidad de casos con place_name_new posterior para analizar: 4780
--------------------------------
Zona
Cantidad de casos con Zona informada: 121220
Cantidad de casos con place_name_new original: 4780
Cantidad de casos con place_name_new posterior para analizar: 0
--------------------------------
Cantidad casos que coinciden entre place_name_new y place_name
True     121197

* Creamos una nueva variable para identificar los casos sobre los que nos falta información de localización y luego analizar si corresponde descartar esos casos

In [7]:
# nueva variable para validar la calidad del dato place_with_parent_names luego de separarlo 
data["calidad_dato"] = False
# dependiendo de la zona que estamos analizando entendemos que la cantidad de info que necesitamos
# para ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur'] el mínimo tendria que ser Localidad
# para  ['Buenos Aires Costa Atlántica'] y en especial MDQ el mínimo tendria que ser Localidad
# para el resto_zonas Partido_barrio
# se va actualizando la variable creada (calidad_dato) a True que identifica los casos sobre los que necesitamos mayor informacion

# para ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']
# zonas a traves de una lista de valores
valores_zona = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']
# mascara sobre la lista de valores y que localidad este nulo o ''
mask_Localidad = ([x in valores_zona for x in data.Zona]) & ((data.Localidad.isnull())|(data.Localidad == ''))
# actualizo la variable creada sobre los True por la mascara
data.loc[mask_Localidad, "calidad_dato"] = True
# ----------------------------------------------------------------------
# caso especial de MDQ
valores_zona_mdq = ['Buenos Aires Costa Atlántica']
mask_Localidad_mdq = ([x in valores_zona_mdq for x in data.Zona])\
& (data.Partido_barrio == 'Mar del Plata') & ((data.Localidad.isnull()) | (data.Localidad == ''))
data.loc[mask_Localidad, "calidad_dato"] = True
# ----------------------------------------------------------------------
#resto_zonas
valores_zona_resto = ['Buenos Aires Costa Atlántica','Buenos Aires Interior','Capital Federal','Resto de provincias']
mask_Partido_barrio = ([x in valores_zona_resto for x in data.Zona]) & ((data.Partido_barrio.isnull())|\
                                                                                (data.Partido_barrio == ''))
mask_calidad_dato_false = data.calidad_dato == False
data.loc[mask_Partido_barrio & mask_calidad_dato_false, "calidad_dato"] = True
# ----------------------------------------------------------------------
# todos los casos con True corresponden a los registros sobre los que nos falta informacion referida a 
# la geolocalización de la propiedad
print ('Cantidad de casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------')
print(data.calidad_dato.value_counts())
# mascara sobre casos que nos faltaria info
mask_faltante = (data.calidad_dato == True)
data_faltante = data.loc[mask_faltante]
#print (data_faltante)
# agrupo por Pais, Zona, Partido_barrio los casos que no tenemos otra info
groupby_df_faltante = data_faltante.groupby(['Pais',  'Zona', 'Partido_barrio'])["Id_caso"].count().\
sort_values(ascending=False)
print ('Agrupación de los casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------------')
groupby_df_faltante


Cantidad de casos a los que le falta información de localización:
-----------------------------------------------------------------
False    107421
True      13799
Name: calidad_dato, dtype: int64
Agrupación de los casos a los que le falta información de localización:
-----------------------------------------------------------------------


Pais       Zona                          Partido_barrio     
Argentina  Bs.As. G.B.A. Zona Norte      Tigre                  2359
                                         Pilar                  1857
           Capital Federal                                      1297
           Bs.As. G.B.A. Zona Norte      Escobar                1037
           Bs.As. G.B.A. Zona Sur        La Plata                767
           Bs.As. G.B.A. Zona Norte      San Isidro              641
           Bs.As. G.B.A. Zona Oeste      Morón                   582
                                         Ituzaingó               474
           Bs.As. G.B.A. Zona Norte      San Fernando            437
           Bs.As. G.B.A. Zona Oeste      Moreno                  434
           Bs.As. G.B.A. Zona Norte      San Miguel              417
                                         General San Martín      382
           Bs.As. G.B.A. Zona Sur        Lomas de Zamora         276
           Bs.As. G.B.A. Zona Oeste      M

  **geonames_id**
  
  Del proceso exploratorio vemos que se corresponde a un Id sobre la variable place_name.  Vamos a:
  * Validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
  * En caso que exista, vamos a definir la tarea de normalización sobre cada uno
  * Por último vamos a completar los valores nulos

*Validacion geonames_id*

In [8]:
# variable geonames_id,  corresponde al id asignado a cada place_name
# primero vamos a validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
mask_geonames_id_nonull = data.geonames_id.notnull()
df_geonames_id_place_name_ag = data.loc[mask_geonames_id_nonull]

grouped = df_geonames_id_place_name_ag.groupby(['geonames_id',  'place_name_new'])
grouped1 = grouped['Id_caso'].agg([np.size])

df_agrupado2 = grouped1.groupby(['geonames_id']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado2)

# vemos que existen 5 valores de geonames_id que tiene mas de un valor de place_name
# por lo que nos vemos en la necesidad de analizar un poco mas, para ver si corresponden a la misma zona
# y ver que trabajo hacemos sobre ellos


Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
                                          size
geonames_id place_name_new                    
3428927.0   San Jose                         5
            San José                        11
3430234.0   Palermo                       2885
            Palermo Soho                   394
3433359.0   Ituzaingó                      832
            Villa Udaondo                   25
3435548.0   Centro                         231
            Centro / Microcentro           223
3435907.0   Bs.As. G.B.A. Zona Norte       222
            Bs.As. G.B.A. Zona Oeste        65
            Bs.As. G.B.A. Zona Sur          24
            Buenos Aires Costa Atlántica    27
            Buenos Aires Interior          106


*Normalización y actualizacion de casos*

In [9]:
print ('Comenzamos con el proceso de análisis y actualización de los los casos:')
print ('-----------------------------------------------------------------------')
print ('geonames_id = 3428927')
print ('----------------------')
# geonames_id = 3428927
    # mascara de id
mask_geonames_id  = data.geonames_id == 3428927
    # traemos la informacion de esos casos 
ver = data.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|','|Argentina|Bs.As. G.B.A. Zona Sur|\
Almirante Brown|San Jose|']
mask_place_with_parent_names  =  ([x in valores for x in data.place_with_parent_names]) 
ver1 = data.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"')
print ('Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|')
print ('Actualizamos la variable Partido_barrio = "Almirante Brown"')
print ('Actualizamos la variable Localidad = "San Jose"')
print ('Actualizamos la variable place_name_new = "San Jose"')
    
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # validamos en google que la localidad de 'San Jose' corresponde al partido de 'Almirante Brown'
    # para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
    # actualizamos la variable Partido_barrio = 'Almirante Brown'
    # actualizamos la variable Localidad = 'San Jose'
    # actualizamos la variable place_name_new = 'San Jose'

# creamos una veriable igual a geonames_id sobre le df para trabajar sobre esa
data['geonames_id_new'] = data.geonames_id
# mascara de geonames_id 
mask_geonames_id  = data.geonames_id == 3428927
mask_place_with_parent_names = data.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Sur|\
Lomas de Zamora|San José|'
# actualizamos el valor de Partido_barrio para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_antes.Partido_barrio.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']] = 'Almirante Brown'
print ('Cantidad de casos actualizados:')
data_despues = data.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_despues.Partido_barrio.value_counts())
# actualizamos el valor de Localidad para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_antes.Localidad.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_despues.Localidad.value_counts())
# actualizamos el valor de place_name_new para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes =  data.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_antes.place_name_new.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_despues.place_name_new.value_counts())

Comenzamos con el proceso de análisis y actualización de los los casos:
-----------------------------------------------------------------------
geonames_id = 3428927
----------------------
|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|    11
|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Brown|San Jose|     5
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3428927.0    16
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"
Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
Actualizamos la variable Partido_barrio = "Almirante Brown"
Actualizamos la variable Localidad = "San Jose"
Actualizamos la variable place_name_new = "San Jose"
Cantidad de casos para actualizar:
Lom

In [10]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3430234')
print ('----------------------')
mask_geonames_id  = data.geonames_id == 3430234
# traemos la informacion de esos casos 
ver = data.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Capital Federal|Palermo|','|Argentina|Capital Federal|Palermo|Palermo Soho|']
mask_place_with_parent_names  =  ([x in valores for x in data.place_with_parent_names]) 
ver1 = data.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo') 
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id_new en nulo para el caso de 'Palermo Soho' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data.place_with_parent_names == '|Argentina|Capital Federal|Palermo|Palermo Soho|'
data_antes =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3430234
----------------------
|Argentina|Capital Federal|Palermo|                 2885
|Argentina|Capital Federal|Palermo|Palermo Soho|     394
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3430234.0    3279
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3430234.0    394
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [11]:
#..................................
# geonames_id =  3433359
    # mascara de id
mask_geonames_id  = data.geonames_id == 3433359
print ('geonames_id = 3433359')
print ('---------------------')
    # traemos la informacion de esos casos 
ver = data.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
mask_place_with_parent_names  =  ([x in valores for x in data.place_with_parent_names]) 
ver1 = data.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Villa Udaondo' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|'
data_antes =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())


geonames_id = 3433359
---------------------
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|                  474
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|        358
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|     25
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3433359.0    857
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3433359.0    25
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [12]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3435548')
print ('---------------------')
mask_geonames_id  = data.geonames_id == 3435548
    # traemos la informacion de esos casos 
ver = data.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|','|Argentina|Capital Federal|Centro / Microcentro|']
mask_place_with_parent_names  =  ([x in valores for x in data.place_with_parent_names]) 
ver1 = data.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Mar del Plata' solo para por simple elección y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data.place_with_parent_names == '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|'
data_antes =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())



geonames_id = 3435548
---------------------
|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|    231
|Argentina|Capital Federal|Centro / Microcentro|                 223
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435548.0    454
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435548.0    231
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [13]:
#..................................
# geonames_id = 3435907
    # mascara de id
print ('geonames_id = 3435907')
print ('---------------------')
valores_geonames_id = [3435907]
mask_geonames_id  = ([x in valores_geonames_id for x in data.geonames_id]) 
    # traemos la informacion de esos casos 
ver = data.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Norte|','|Argentina|Bs.As. G.B.A. Zona Oeste|','|Argentina|Bs.As. G.B.A. Zona Sur|','|Argentina|Buenos Aires Costa Atlántica|','|Argentina|Buenos Aires Interior|']
mask_place_with_parent_names  =  ([x in valores for x in data.place_with_parent_names]) 
ver1 = data.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
data_antes =   data.loc[mask_geonames_id, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data.loc[mask_geonames_id, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data.loc[mask_geonames_id, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3435907
---------------------
|Argentina|Bs.As. G.B.A. Zona Norte|        222
|Argentina|Buenos Aires Interior|           106
|Argentina|Bs.As. G.B.A. Zona Oeste|         65
|Argentina|Buenos Aires Costa Atlántica|     27
|Argentina|Bs.As. G.B.A. Zona Sur|           24
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435907.0    444
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435907.0    444
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


*Asignación de valores sobre casos nulos*

In [14]:
import random
print('Variable geonames_id')
print('Vamos a analizar como podemos actualizar los valores nulos')
print('----------------------------------------------------------')
print('Cantidad de valores nulos: ' + str(data.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos: ' + str(len(data.geonames_id_new.unique())))
print('Valor mínimo: ' + str(data.geonames_id_new.min()))
print('Valor máximo: ' + str(data.geonames_id_new.max()))
# vamos a trabajar los nulos del campo geonames_id_new\n# y lo hacemos por zonas:
# definimos 3 zonas de acuerdo al analisis del dato faltante de los procesos sobre place_with_parent_names
# valores_geo_grupo1  = \'Bs.As. G.B.A. Zona Norte\',\'Bs.As. G.B.A. Zona Oeste\',\'Bs.As. G.B.A. Zona Sur\',\'Capital Federal\'\n# mas valores_geo_interior\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio asigamos un numero al azar que identificara el place_name\n
# valores_geo_grupo2 = \'Buenos Aires Costa Atlántica\', \'Buenos Aires Interior\'\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio y Localidad, y NO son iguales asigamos un numero al azar que identificara el place_name\n    
    # si tiene Partido_barrio y Localidad, y SI son iguales asigamos, buscamos el ID del Partido_barrio \n    
# que si está informado\n\n
# creamos los grupos\n
valores_geo_grupo1 = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur','Capital Federal']
valores_geo_grupo2 = ['Buenos Aires Costa Atlántica', 'Buenos Aires Interior']
valores_geo_interior_temp = data.Zona.unique()
valores_geo_interior_y_1 = list(set(valores_geo_interior_temp).difference(set(valores_geo_grupo2)))

mask_geo_grupo2  =  ([x in valores_geo_grupo2 for x in data.Zona])
mask_geo_interior_y_1  =  ([x in valores_geo_interior_y_1 for x in data.Zona])
mask_geo_null = data.geonames_id_new.isnull()
#mask_Partido_barrio_notnull = data.Partido_barrio.notnull()
mask_Partido_barrio_null =  (data.Partido_barrio.isnull()) | (data.Partido_barrio == '')
mask_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
mask_PB_dist_Localidad = data.Partido_barrio != data.Localidad

# Creamos una serie con los distintos valores de place_name que no interesa aplicarle una numero al azar
groupby_place_name_null = data.loc[(mask_geo_interior_y_1 & mask_geo_null & mask_Partido_barrio_notnull)\
                                   | (mask_geo_grupo2&mask_geo_null&mask_Partido_barrio_notnull&mask_PB_dist_Localidad)\
                                   ].groupby(['place_name_new'])["Id_caso"].count()
# convertimos la seria en un df para trabajarlo
groupby_place_name_null_df = groupby_place_name_null.to_frame()
# asignamos a cada valor el número aleatorio partiendo de un numero lo bastante grande para que
# coincida con los existentes
groupby_place_name_null_df["geonames_id_temp"] = groupby_place_name_null_df.apply(lambda x: random.randint(10000000, 20000000), axis=1)

# creamos un nuevo df sumando el campo nuevo 
datay = pd.merge(left=data,right=groupby_place_name_null_df, how='left', left_on='place_name_new', \
                 right_on='place_name_new')
#-------------------------------------------------------------------------------------------------------------
# actualizamos los casos que convertimos a nulo en el arreglo de geonames_id
#mask_geo_temp = ['|Argentina|Capital Federal|Palermo|Palermo Soho|']
#mask_geo_temp1  =  ([x in mask_geo_temp for x in datay.place_with_parent_names])
#max_valor = datay.geonames_id_temp.max()
#max_valor = max_valor + 1
#print(max_valor)
#datay.loc[mask_geo_temp1, 'geonames_id_temp'] = datay.loc[mask_geo_temp1, 'geonames_id_temp'].apply(lambda x: max_valor)
#['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|']
#['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
#-------------------------------------------------------------------------------------------------------------

mask_geo_null = data.geonames_id_new.isnull()
datay.loc[mask_geo_null, 'geonames_id_new'] = datay.loc[mask_geo_null, 'geonames_id_temp']
del datay['Id_caso_y']
datay.rename(columns={'Id_caso_x': 'Id_caso'}, inplace=True)
print('Cantidad de valores nulos final: ' + str(datay.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos final: ' + str(len(datay.geonames_id_new.unique())))

#Validamos que luego de la aplicacion de nulos no hayan quedado para un mismo 
# geonames_id a distintos valores de place_name_new

# variable geonames_id,  corresponde al id asignado a cada place_name
# volvemos a validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
mask_geonames_id_new_notnull = datay.geonames_id_new.notnull()
df_geonames_id_place_name_ag2 = datay.loc[mask_geonames_id_new_notnull]

grouped_0 = df_geonames_id_place_name_ag2.groupby(['geonames_id_new',  'place_name_new'])
grouped_1 = grouped_0['Id_caso'].agg([np.size])

df_agrupado_2 = grouped_1.groupby(['geonames_id_new']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado_2)

data = datay
data.drop("geonames_id_temp", axis = 1, inplace=True)

print ('Casos con geonames_id nulo')
print ('--------------------------')
# actualizo la mascara de los nulos
mask_geo_null = data.geonames_id_new.isnull()
print (data.loc[mask_geo_null, 'place_with_parent_names'].value_counts())
print ('-----------------------------------------------------------------------------------')
print ('Conclusión:')
print ('Se terminó asignado un valor al azar mayor al máximo valor de la variable original')
print ('solo quedando 610 casos nulos, que junto con la variable creada de calidad de dato,')
print ('nos ayuda para definir o no su eliminación')

Variable geonames_id
Vamos a analizar como podemos actualizar los valores nulos
----------------------------------------------------------
Cantidad de valores nulos: 19811
Cantidad de valores distintos: 646
Valor mínimo: 3427208.0
Valor máximo: 6948895.0
Cantidad de valores nulos final: 610
Cantidad de valores distintos final: 1107
Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
Empty DataFrame
Columns: [size]
Index: []
Casos con geonames_id nulo
--------------------------
|Argentina|Bs.As. G.B.A. Zona Norte|                                     222
|Argentina|Buenos Aires Interior|                                        106
|Argentina|Tucumán|                                                       77
|Argentina|Bs.As. G.B.A. Zona Oeste|                                      65
|Argentina|Buenos Aires Interior|Chascomús|Chascomús|                     54
|Argentina|Buenos Aires Costa Atlántica|           

  **Lat-Lon /** 
  **Lat /**
  **Lon /**
  
  Vamos a validar que Lat-Lon corresponde a la concatenación de las variables Lat y Lon, con el objetivo de descartar las variables repetidas

In [15]:
print ('Proceso de validación de los campos Lat-Lon vs Lat / Lon')
print ('--------------------------------------------------------')
# validamos que los campos Lat y Lon tengan la misma cantidad y valor de info que Lat-Lon
# primero separamos el campo Lat-Lon
df_lat_lon  = data.loc[:,["Id_caso","lat-lon", "lat", "lon"]].join(data["lat-lon"].str.split(',', expand=True).rename(columns={
                                                                                                0:'lat_temp', 
                                                                                                1:'lon_temp'}))
# los campos saparados los convierto en numéricos
df_lat_lon.lat_temp = df_lat_lon.lat_temp.apply(pd.to_numeric)
df_lat_lon.lon_temp = df_lat_lon.lon_temp.apply(pd.to_numeric)

# Validmos que los campos tengan la misma cantidad de NO nulos
print (df_lat_lon[["lat-lon", "lat", "lon"]].notnull().sum())
# mascara de no nulos
mask_notnull = df_lat_lon["lat-lon"].notnull()
# mascara de valores iguales 
ver_lat = round(df_lat_lon.lat,6) == round(df_lat_lon.lat_temp,6)
ver_lon = round(df_lat_lon.lon,6) == round(df_lat_lon.lon_temp,6)
# validamos la cantidad de casos
print ('Cantidad de casos iguales latitud: ' + str(ver_lat.sum()))
print ('Cantidad de casos iguales longitud: ' + str(ver_lon.sum()))
print ('----------------------------------------------------------')
print ('Se validó que la variable lat-lon corresponde a la concatenación')
print ('de las variables individuales')

Proceso de validación de los campos Lat-Lon vs Lat / Lon
--------------------------------------------------------
lat-lon    69670
lat        69670
lon        69670
dtype: int64
Cantidad de casos iguales latitud: 69502
Cantidad de casos iguales longitud: 69396
----------------------------------------------------------
Se validó que la variable lat-lon corresponde a la concatenación
de las variables individuales


---
# 2.  Variables del grupo sobre el valor de propiedades

Validación, limpieza y asignación de nulos; sobre las variables:

- price
- currency
- price_aprox_local_currency
- price_aprox_usd
- price_usd_per_m2
- price_per_m2

**price /**
**currency /**
**price_aprox_local_currency /**
**price_aprox_usd**

Validamos si la variable price_aprox_usd es la podemos utilizar como precio estandar de una propiedad:
* Analizamos si para todos los casos está asignado el mismo tipo de cambio por dolar, independientemente de la moneda.

In [16]:
print ('Proceso de validación de tipo de cambio')
print ('---------------------------------------')
# proceso de validación del tipo de cambio para validar si es correcto tomar el 
# el campo price_aprox_usd para todos los casos

# validamos que si currency (moneda) = 
# nulo  ----> nulo
# 'USD'  ----> price_aprox_local_currency / price  ----> valor por dolar de precios en dolares
# sino ---->price_aprox_local_currency / price_aprox_usd ----> valor por dolar de precios en pesos
# agrupamos los valores por dolar ... redondeado a 2 dígitos

#.................................................
print ('Distintos tipos de moneda:')
# Vemos los distintos valores de la variable moneda
print(data.currency.value_counts())
# vemos que existen 3 registros con valores PEN (sol peruano) y UYU (peso Uruguayo)
# vemos los casos
print ('Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos')
valores_otras_monedas = ['PEN', 'UYU']
mask_otras_monedas = [x in valores_otras_monedas for x in data.currency]
data.loc[mask_otras_monedas, ["currency","price"]] = None

# sumamos la varible al Df Valor_dolar
data["Valor_dolar"] = 0
# trabajamos la moneda = 'USD'
mask_dolar = data.currency == 'USD'
mask_price = (data.price != 0) | (data.price.notnull())
data.loc[mask_dolar & mask_price, "Valor_dolar"] = round(data.price_aprox_local_currency / data.price,2)
# trabajamos la moneda = 'ARG'
mask_peso = data.currency == 'ARS'
mask_price_aprox_usd = (data.price_aprox_usd != 0) | (data.price_aprox_usd.notnull())
data.loc[mask_peso & mask_price_aprox_usd, "Valor_dolar"] = round(data.price_aprox_local_currency / data.price_aprox_usd,2)
print ('Distintos valores del tipo de cambio luego de actualizar la variable creada')
# vemos los valores distintos para valor de dolar
print(data.Valor_dolar.value_counts())
print ('Vemos que existe un solo valor ... Exc!')
print ('-------------------------------------------------------------------------------------------------------------')
print ('Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar')

Proceso de validación de tipo de cambio
---------------------------------------
Distintos tipos de moneda:
USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64
Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos
Distintos valores del tipo de cambio luego de actualizar la variable creada
17.64    100806
0.00      20414
Name: Valor_dolar, dtype: int64
Vemos que existe un solo valor ... Exc!
-------------------------------------------------------------------------------------------------------------
Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar


**price_usd_per_m2 /**
**price_per_m2**


* Análisis sobre las variables
* Estandarización sobre price_per_m2 a precio u$s
* Lógica sobre valores nulos de price_per_m2

In [17]:
print ('Proceso de análisis del precio por m2')
print ('-------------------------------------')
print('Variables:')
print('- price_usd_per_m2')
print('- price_per_m2')
    
print ('Al ver alguno ejemplos vamos a validar que:')
print ('price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)')
print ('y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)')
print ('por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable')
    

# lo validamos
print ('Validación de price_usd_per_m2')
print ('------------------------------')
# ----------------------------------price_usd_per_m2
# sumamos un nuevo campo 
data["price_usd_per_m2_new"] = -1
# mascara de los NO nulos price_aprox_usd
mask_price_aprox_usd_notnull = data.price_aprox_usd.notnull()
# mascara de los no nulos y mayores a cero de surface_total_in_m2
mask_surface_total_in_m2_notnull_0 = ((data.surface_total_in_m2.notnull()) & (data.surface_total_in_m2 > 0))
# actualización de la variable nueva price_usd_per_m2_new
data.loc[mask_price_aprox_usd_notnull & mask_surface_total_in_m2_notnull_0, "price_usd_per_m2_new"] = round(data.price_aprox_usd/data.surface_total_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_aprox_usd = data.price_usd_per_m2_new != round(data.price_usd_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 52606 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_aprox_usd.value_counts())
print('Vemos que hay 52606 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print(data.loc[mask_validacion_price_aprox_usd, "price_usd_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_usd_per_m2 corresponde a')
print('a price_aprox_usd / surface_total_in_m2')

# ----------------------------------price_per_m2
print ('Validación de price_per_m2')
print ('------------------------------')
# sumamos un nuevo campo 
data["price_per_m2_new"] = -1
# mascara de los NO nulos price
mask_price_notnull = data.price.notnull()
# mascara de los no nulos y mayores a cero de surface_covered_in_m2
mask_surface_covered_in_m2_notnull_0 = ((data.surface_covered_in_m2.notnull()) & (data.surface_covered_in_m2 > 0))
# actualización de la variable nueva price_per_m2_new
data.loc[mask_price_notnull & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data.price/data.surface_covered_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_per_m2 = data.price_per_m2_new != round(data.price_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 33562 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_per_m2.value_counts())
print('Vemos que hay 33562 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print (data.loc[mask_validacion_price_per_m2, "price_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_per_m2 corresponde a')
print('price / surface_covered_in_m2')
# ahora vamos a actualizar la veriable que creamos price_per_m2 pero tomando para el caso de moneda ARG la 
# variable en dolares price_aprox_usd para que todo quede en la misma moneda
# mascara moneda ARG
mask_peso = data.currency == 'ARS'
data.loc[mask_peso & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data.price_aprox_usd/data.surface_covered_in_m2,2)
# pasamos a nulo los que quedaron con valor -1
data['price_usd_per_m2_new'].replace(-1, np.nan, inplace=True)
data['price_per_m2_new'].replace(-1, np.nan, inplace=True)


Proceso de análisis del precio por m2
-------------------------------------
Variables:
- price_usd_per_m2
- price_per_m2
Al ver alguno ejemplos vamos a validar que:
price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)
y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)
por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable
Validación de price_usd_per_m2
------------------------------
Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:
False    68614
True     52606
dtype: int64
Vemos que hay 52606 que no coinciden
Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara
True     52603
False        3
Name: price_usd_per_m2, dtype: int64
Conclusión: la variable price_usd_per_m2 corresponde a
a price_aprox_usd / surface_total_in_m2
Validación de price

In [18]:
print ('Proceso para llenar nulos sobre price_per_m2_new')
print ('------------------------------------------------')

print ('Cantidad de casos nulos: ' + str(data.price_per_m2_new.isnull().sum()))

print ('Tomamos la decision de reemplazar los nulos por:')
print ('data.price_aprox_usd / data.surface_total_in_m2')
print ('Pero para los casos que:')
print ('property_type == "apartment"')
print ('price_aprox_usd no sea nulo')
print ('surface_total_in_m2 no sea nulo y mayor a cero')

# mascaras necesarias
mask_property_type = data.property_type == 'apartment'
mask_price_aprox_usd = data.price_aprox_usd.notnull()
mask_surface_total_in_m2 = data.surface_total_in_m2.notnull()&data.surface_total_in_m2 > 0
mask_surface_covered_in_m2 = data.surface_covered_in_m2.isnull()
mask_price_per_m2_new = data.price_per_m2_new.isnull()

data.loc[mask_property_type&mask_price_aprox_usd&\
                 mask_surface_total_in_m2&mask_surface_covered_in_m2&mask_price_per_m2_new, "price_per_m2_new"] =\
data.price_aprox_usd /  data.surface_total_in_m2 

print ('Cantidad de casos actualizados: ' + str(len(data.loc[mask_property_type&mask_price_aprox_usd&\
                 mask_surface_total_in_m2&mask_surface_covered_in_m2, "price_per_m2_new"] )))
print ('------------------------------------------------')
print ('Cantidad de casos nulos despues del proceso: ' + str(data.price_per_m2_new.isnull().sum()))

Proceso para llenar nulos sobre price_per_m2_new
------------------------------------------------
Cantidad de casos nulos: 33562
Tomamos la decision de reemplazar los nulos por:
data.price_aprox_usd / data.surface_total_in_m2
Pero para los casos que:
property_type == "apartment"
price_aprox_usd no sea nulo
surface_total_in_m2 no sea nulo y mayor a cero
Cantidad de casos actualizados: 3309
------------------------------------------------
Cantidad de casos nulos despues del proceso: 30253


---
# 3.  Proceso de borrado de registros duplicados

Luego del analisis de cada uno de los datos, llegamos a la conclusion de ejecutar el borrado de los datos duplicados sobre las variables:
* property_type
* geonames_id_new
* lat-lon
* price_aprox_usd
* price_per_m2_new
* description
* title

Teniendo en cuenta como afecta al proceso la cantidad de nulos de cada variable

In [19]:
print ('Proceso de borrado de duplicados')
print ('--------------------------------')

print('Cantidad de registros con duplicados: ' + str(len(data)))

# analizando las varibles disponibles decidimos validar la existencia de duplicados por:
# property_type:    no existen nulos
# geonames_id_new:  no existen nulos
# lat-lon:          existen nulos
# price_aprox_usd:  existen nulos
# price_per_m2_new: existen nulos
# description:      no existen nulos
# title:            no existen nulos

print ('Cantidad de registros duplicados: ' + str(data.duplicated(subset=[\
                                                                                  "property_type",\
                                                                                  "geonames_id_new",\
                                                                                  "lat-lon",\
                                                                                  "price_aprox_usd",\
                                                                                  "price_per_m2_new",\
                                                                                  "description",\
                                                                                  "title"\
                                                                                 ], keep="last").sum())\
      )

data.drop_duplicates(subset=[\
                                     "property_type",\
                                     "geonames_id_new",\
                                     "lat-lon",\
                                     "price_aprox_usd",\
                                     "price_per_m2_new",\
                                     "description",\
                                     "title"\
                                    ], keep="last", inplace=True)

print('Cantidad de registros sin duplicados: ' + str(len(data)))

Proceso de borrado de duplicados
--------------------------------
Cantidad de registros con duplicados: 121220
Cantidad de registros duplicados: 6169
Cantidad de registros sin duplicados: 115051


---
# 4.  Proceso de borrado de registros sin información completa

De acuerdo al trabajo de limpieza que generamos, vemos que existen casuísticas que debemos tener en cuenta y que debemos descartar

In [20]:
print ('Planteamos tres escenarios que nos permiten definifir los conjuntos de datos que pueden ser borrados:')
print ('-----------------------------------------------------------------------------------------------------')
print ('Cantidad de registros original: ' + str(len(data)))
print ('1. price_aprox_usd es null & surface_covered_in_m2 es null & price_per_m2_new is null')
maskf_price_aprox_usd = data.price_aprox_usd.isnull()
maskf_surface_covered_in_m2 = data.surface_covered_in_m2.isnull()
maskf_price_per_m2_new = data.price_per_m2_new.isnull()
print (len(data.loc[maskf_price_aprox_usd&maskf_surface_covered_in_m2&maskf_price_per_m2_new]))
data.drop(data[maskf_price_aprox_usd&maskf_surface_covered_in_m2&maskf_price_per_m2_new].index, inplace=True)

print ('2. calidad_dato = True & lat-lon es null')
maskf_calidad_dato = data.calidad_dato == True
maskf_lat_lon = data["lat-lon"].isnull()
print (len(data.loc[maskf_calidad_dato&maskf_lat_lon]))
data.drop(data[maskf_calidad_dato&maskf_lat_lon].index, inplace=True)

print ('3. aquellas regiones con una cantidad de casos muy chica')
list_state_name = ['Tierra Del Fuego','Catamarca','Jujuy','Santa Cruz','La Rioja','Santiago Del Estero']
mask_state_name = [x in list_state_name for x in data.state_name]
print (len(data.loc[mask_state_name]))
data.drop(data[mask_state_name].index, inplace=True)
print ('-----------------------------------------------------------------------------------------------------')
print ('Eliminamos los registros:')
#data.drop(data[maskf_price_aprox_usd&maskf_surface_covered_in_m2&maskf_price_per_m2_new].index, inplace=True)
#data.drop(data[maskf_calidad_dato&maskf_lat_lon].index, inplace=True)
#data.drop(data[mask_state_name].index, inplace=True)
print ('-----------------------------------------------------------------------------------------------------')
print ('Validamos el resultado:')
print ('Cantidad de registros final: ' + str(len(data)))


Planteamos tres escenarios que nos permiten definifir los conjuntos de datos que pueden ser borrados:
-----------------------------------------------------------------------------------------------------
Cantidad de registros original: 115051
1. price_aprox_usd es null & surface_covered_in_m2 es null & price_per_m2_new is null
5689
2. calidad_dato = True & lat-lon es null
7041
3. aquellas regiones con una cantidad de casos muy chica
93
-----------------------------------------------------------------------------------------------------
Eliminamos los registros:
-----------------------------------------------------------------------------------------------------
Validamos el resultado:
Cantidad de registros final: 102228


In [21]:
df_final = data.loc[:, [\
                            'Id_caso','property_type','place_name_new',\
                            'state_name','geonames_id_new','lat-lon', 'calidad_dato',\
                            'price_aprox_usd',\
                            'surface_total_in_m2','surface_covered_in_m2',\
                            'price_usd_per_m2','price_per_m2_new',\
                            'rooms',\
                            'description','title']]

In [22]:
df_final.corr().price_per_m2_new

Id_caso                 -0.016277
geonames_id_new          0.003905
calidad_dato            -0.000373
price_aprox_usd          0.099110
surface_total_in_m2     -0.011256
surface_covered_in_m2   -0.010896
price_usd_per_m2         0.455498
price_per_m2_new         1.000000
rooms                   -0.021756
Name: price_per_m2_new, dtype: float64